In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt


In [2]:
train_data = pd.read_csv(r'fashion-mnist_train.csv')
train_data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test_data = pd.read_csv(r'fashion-mnist_test.csv')
test_data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,8,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,8,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
label_column_name = train_data.columns[0]
pixel_column_names = train_data.columns[1:]
y = train_data[label_column_name]
x = train_data[pixel_column_names]

In [5]:
y_test = test_data[label_column_name]
x_test = test_data[pixel_column_names]

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [7]:
x_train_tensor = torch.IntTensor(x_train.to_numpy())
x_val_tensor = torch.IntTensor(x_val.to_numpy())
y_train_tensor = torch.IntTensor(y_train.to_numpy())
y_val_tensor = torch.IntTensor(y_val.to_numpy())



In [8]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=256, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=20, drop_last=True)

In [9]:
import torch.nn as nn

In [10]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=2)
        self.activation_func = nn.ReLU()
        
        self.linear_layer1 = nn.Linear(4*4*32, 200)
        self.linear_layer2 = nn.Linear(200, 100)
        self.linear_layer3 = nn.Linear(100, 50)
        self.linear_layer4 = nn.Linear(50, 10)




        
    def forward(self, x):
        x = x.view(20, 1, 28, 28).type(torch.FloatTensor)
        
        x = self.conv1(x)
        x = self.activation_func(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.activation_func(x)
        x = self.pool2(x)
                
        x = x.view(-1, 4 * 4 * 32).type(torch.FloatTensor)
        x = self.linear_layer1(x)
        x = self.activation_func(x)
        x = self.linear_layer2(x)
        x = self.activation_func(x)
        x = self.linear_layer3(x)
        x = self.activation_func(x)
        x = self.linear_layer4(x)
        


        return x
        

In [11]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

epoches_num = 10

In [12]:
for epoch in range(epoches_num):
    print(f'Epoch {epoch}:')
    for c, (x_batch, y_batch) in enumerate(train_dataloader):
        y_batch = y_batch.type(torch.LongTensor)

        outp = model(x_batch)
        preds = outp.argmax(-1)

        loss = criterion(outp, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (c%100==0) or (c == len(train_dataloader)-1):
            print('\t' + str(accuracy_score(preds, y_batch)))


Epoch 0:


RuntimeError: shape '[20, 1, 28, 28]' is invalid for input of size 200704

In [ ]:
model.eval()
with torch.no_grad():
    acc_list = []
    for x_batch, y_batch in val_dataloader:
        outp = model(x_batch)
        preds = outp.argmax(-1)
        acc = accuracy_score(preds, y_batch)
        acc_list.append(acc)
    
        
        

In [ ]:
plt.figure(figsize=(16, 10))
plt.title("Valid accuracy")
plt.plot(range(len(acc_list)), acc_list, label="ReLU activation", linewidth=2)

In [ ]:
np.array(acc_list).mean()

In [ ]:
corres = {
0: 'T-shirt/top',
1: 'Trouser',
2: 'Pullover',
3: 'Dress',
4: 'Coat',
5: 'Sandal',
6: 'Shirt',
7: 'Sneaker',
8: 'Bag',
9: 'Ankle boot'
}

In [ ]:
x_batch, y_batch = next(iter(val_dataloader))
model.eval()
with torch.no_grad():
    outp = model(x_batch)
    preds = outp.argmax(-1)[:10]
for i in range(10):
    plt.subplot(2, 5, i+1)
    numpy_img = x_batch[i].numpy().reshape(28, 28)
    plt.title(corres[int(preds[i])] + '\n' + corres[int(y_batch[i])])
    plt.imshow(numpy_img, cmap='gray');